<a href="https://colab.research.google.com/github/hdakhli/mlops-2024/blob/main/30_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#
## Word Embeddings

In [ ]:
!pip install nltk
!pip install gensim

# Download EBOOK ALICE'S ADVENTURES IN WONDERLAND
# https://www.gutenberg.org/files/11/11-0.txt

In [ ]:
# Lire le fichier 'alice.txt'

In [ ]:
# Remplacer les retours à la ligne \n par des espaces

In [ ]:
# Extraire chaque phrase et les mots qui la compose en un tableau de tableau à l'aide des fonctions sent_tokenize et word_tokenize de nltk.tokenize

In [ ]:
# Créer 2 modèles: CBOW et Skip Gram à partir de ce tableau. cf doc: gensim.models.Word2Vec

In [ ]:
# Comparer les mots suivants avec les 2 modelès : 
# 'alice' vs 'wonderland'
# 'alice' vs 'machines'

#
#
## LLM




In [ ]:
# Download: https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/blob/main/llama-2-7b-chat.Q2_K.gguf

In [ ]:
!pip install llama-cpp-python

In [ ]:
from llama_cpp import Llama

# Put the location of to the GGUF model that you've download from HuggingFace here
model_path = "llama-2-7b-chat.Q2_K.gguf"

# Create a model
model = Llama(model_path=model_path)

In [ ]:
# Prompt creation
system_message = "Vous êtes un assistant utile. Vous êtes là pour m'aider avec mes questions"
user_message = "Quelle est la capitale de la France ?"

prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{user_message} [/INST]"""

# Model parameters
max_tokens = 50

# Run the model
output = model(prompt, max_tokens=max_tokens, echo=True)

# Print the model output
print(output)

In [ ]:
user_message = "Et celle de l'Espagne ? "
prompt = f"""<s>[INST] <<SYS>>
{system_message}
<</SYS>>
{user_message} [/INST]"""

output = model(prompt, max_tokens=max_tokens, echo=True)

print(output)

In [ ]:
# Que remarquez-vous ?
# Comment pourriez-vous résoudre le problème ?
# Exposer ce model en API REST avec FastAPI et Uvicorn

### LangChain

In [ ]:
!pip install langchain-core langchain-community

In [ ]:
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q2_K.gguf", verbose=False
)

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Vous êtes un assistant utile. Vous êtes là pour répondre à mes questions {input}.",
        )
    ]
)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"Quelle est la capitale de la France ?"})

In [ ]:
chain.invoke({"Et celle de l'Espagne ?"})

In [ ]:
# Faites en sorte que le modèle prenne en compte l'historique de la conversation
# Exposer le model en API à l'aide de fast api et langserve